In [ ]:
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")

import numpy as np
from visu import Visualizer
import imageio


# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches

In [ ]:
from PIL import Image
import cv2
import torch
from torchvision import transforms as tf
from utils_asl import LabelLoaderAuto
from pathlib import Path
import copy
import numpy as np
import cv2

In [ ]:
def get_labels(p,sub=1,la="png"):
    p = [str(s) for s in Path(p).rglob(f'*.{la}') if str(s).find("_.png") == -1]
    
    p.sort(
        key=lambda x: int(x.split("/")[-1][:-4])
    )
    return p[::sub]



In [ ]:
import torch
kimera_interfacer_path = "/home/jonfrey/catkin_ws/src/Kimera-Interfacer/kimera_interfacer"
mapping = np.genfromtxt(
  f"{kimera_interfacer_path}/cfg/nyu40_segmentation_mapping.csv", delimiter=","
)
rgb = torch.from_numpy(mapping[2:, 1:4]).type(torch.float32)

filt = np.array( [255**2, 255**1,1])[None,:].repeat(rgb.shape[0],0)
ind = (rgb * filt).sum(axis= 1)

In [14]:
W,H = 640,480
path_labels = "/home/jonfrey/Datasets/labdata/result/full_run_2/pseudo_label"
path_visu = "/home/jonfrey/Datasets/labdata/result/full_run_2/pseudo_label".replace("pseudo_label", "visu")

visu = Visualizer(path_visu, logger=None, epoch=0, store=True, num_classes=41)
lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet", confidence= 0.1,H = 480, W = 640)

labels_p = get_labels(path_labels)
img_p = [p.replace("pseudo_label","color").replace(".png",".jpg") for p in labels_p]

filt = np.array( [255**2, 255**1,1])[None,None,:].repeat(480,0).repeat(640,1)

# VIDEO
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
FPS = 30
fourcc = VideoWriter_fourcc("M", "J", "P","G")
video = VideoWriter(path_visu+ "/conference_video_pseudo.avi", fourcc, float(FPS), (W,H))
fourcc2 = VideoWriter_fourcc("M", "J", "P","G")
video2 = VideoWriter(path_visu+ "/conference_video_network.avi", fourcc2, float(FPS), (W,H))


for i in range(len(labels_p)):
    visu.epoch = int(labels_p[i].split('/')[-1][:-4])
    gt = lla.get(labels_p[i])[0]
    img = imageio.imread(img_p[i]) 
    
    img2 = imageio.imread(labels_p[i].replace("pseudo_label","label-filt"))
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)  
    
    label = (img2* filt ).sum(axis=2)
    out = np.zeros_like(label)
    
    for i in range(0,ind.shape[0]):
        out[ label == int( ind[i]) ] = i+1
    
    visu.plot_image(img2, tag ="conference_room_image", jupyter=False)
    res2 = visu.plot_detectron( img , out, tag="conference_network", jupyter=False,text_off=True )
    res = visu.plot_detectron( img , gt, tag="conference_pseudo", jupyter=False, text_off=True)
    
    img = Image.fromarray(res)
    r, g, b = img.split()
    img = np.array( Image.merge("RGB", (b, g, r)) )
    video.write(img)
    
    img = Image.fromarray(res2)
    r, g, b = img.split()
    img = np.array( Image.merge("RGB", (b, g, r)) )
    video2.write(img)
    if i == 10: break
video.release()

In [ ]:

filt = np.array( [255**2, 255**1,1])[None,None,:].repeat(480,0).repeat(640,1)
visu = Visualizer("/home/jonfrey/Documents/master_thesis/Report/images/labdata", logger=None, epoch=0, store=True, num_classes=41)
lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet", confidence= 0.1,H = H, W = W)

import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
FPS = 30
fourcc = VideoWriter_fourcc("M", "J", "P","G")
video = VideoWriter(f'/home/jonfrey/Videos/hypersim_labe_{s}.avi', fourcc, float(FPS), (W,H))

    
for i in  range( len( labels_p) ):
    visu.epoch = int(labels_p[i].split('/')[-1][:-4])
    gt = lla.get(labels_p[i])[0]
    img = imageio.imread(img_p[i]) 
    
    img2 = imageio.imread(labels_p[i].replace("pseudo_label","label-filt"))
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)  
    
    label = (img2* filt ).sum(axis=2)
    out = np.zeros_like(label)
    
    for i in range(0,ind.shape[0]):
        out[ label == int( ind[i]) ] = i+1
    
    #visu.plot_image(img2, tag ="conference_room_image", jupyter=True)
    #visu.plot_detectron( img , out, tag="conference_network", jupyter=True,text_off=True )
    
    ps = visu.plot_detectron( img , gt, tag="conference_pseudo", jupyter=True, text_off=True)

    img = Image.fromarray(ps)
    
    r, g, b = img.split()
    img = np.array( Image.merge("RGB", (b, g, r)) )
    video.write(img)

        
video.release()

In [ ]:
# ScanNet
for s in ["scene0000_00", "scene0001_00", "scene0002_00", "scene0003_00"]:

    labels_p = get_labels(f"/home/jonfrey/Datasets/scannet/scans/{s}/label-filt")
    img_p = [p.replace("label-filt","color").replace(".png",".jpg") for p in labels_p]
    labels_p[:10]

    from cv2 import VideoWriter, VideoWriter_fourcc
    import cv2
    FPS = 30
    fourcc = VideoWriter_fourcc("M", "J", "P","G")
    video = VideoWriter(f'/home/jonfrey/Videos/hypersim_labe_{s}.avi', fourcc, float(FPS), (W,H))
    
    fourcc2 = VideoWriter_fourcc("M", "J", "P","G")
    video2 = VideoWriter(f'/home/jonfrey/Videos/hypersim_images_{s}.avi', fourcc2, float(FPS), (W,H))
    
    
    filt = np.array( [255**2, 255**1,1])[None,None,:].repeat(H,0).repeat(W,1)
    visu = Visualizer("/home/jonfrey/Documents/master_thesis/Report/images/labdata", logger=None, epoch=0, store=False, num_classes=41)


    import cv2
    for i in  range( len( labels_p) ):
        visu.epoch = int(labels_p[i].split('/')[-1][:-4])
        gt = lla.get(labels_p[i])[0]
        img = imageio.imread(img_p[i]) 
        t = np.copy(img)
        #label = (img2* filt ).sum(axis=2)
        #out = np.zeros_like(label)

        #for i in range(0,ind.shape[0]):
        #    out[ label == int( ind[i]) ] = i+1

        #visu.plot_image(img2, tag ="conference_room_image", jupyter=True)

        #visu.plot_detectron( img , out, tag="conference_network", jupyter=True,text_off=True )
        ps = visu.plot_detectron( img , gt, tag="conference_pseudo", jupyter=False, text_off=True)

        img = Image.fromarray(ps)

        r, g, b = img.split()
        img = np.array( Image.merge("RGB", (b, g, r)) )
        video.write(img)
        
        img = Image.fromarray(t)

        r, g, b = img.split()
        img = np.array( Image.merge("RGB", (b, g, r)) )
        video2.write(img)
        
    video2.release()    
    video.release()
